<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_OLAP_Cubes_with_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

OLAP Cubes <br>
https://docs.google.com/presentation/d/1rx_ek2E1F5MNberY2bxeRuEfPOeuVM_WcdAKyAbF_EQ/edit?usp=sharing <br>
[Bigger Slide Deck](https://docs.google.com/presentation/d/1wQzZoHFo3NVk-dMTebHA4LH7B4KYmOfiSammjsSRGL0/edit?usp=sharing)

In [32]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2023-09-02 11:18:12.278971+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy


#Install Spark

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark
# -------------------------------------------------
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
# ----------------------
#import findspark
#findspark.init()

In [1]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


<SparkContext master=local[*] appName=pyspark-shell>

#Car Sales Data ( Small )

##Load Data

In [2]:
# https://drive.google.com/file/d/1JBGx9yMxpHrUO5BUmr12TpDDwLGLmal-/view
!gdown "https://drive.google.com/uc?id=1JBGx9yMxpHrUO5BUmr12TpDDwLGLmal-"

Downloading...
From: https://drive.google.com/uc?id=1JBGx9yMxpHrUO5BUmr12TpDDwLGLmal-
To: /content/carsales2-noHeader.txt
100% 576/576 [00:00<00:00, 1.89MB/s]


##Process

In [3]:
!cat carsales2-noHeader.txt

Honda,SUV,Red,31
Honda,SUV,Blue,46
Honda,SUV,Green,48
Honda,Sedan,Red,25
Honda,Sedan,Blue,41
Honda,Sedan,Green,34
Honda,Compact,Red,48
Honda,Compact,Blue,26
Honda,Compact,Green,32
Toyota,SUV,Red,39
Toyota,SUV,Blue,45
Toyota,SUV,Green,50
Toyota,Sedan,Red,41
Toyota,Sedan,Blue,28
Toyota,Sedan,Green,21
Toyota,Compact,Red,44
Toyota,Compact,Blue,50
Toyota,Compact,Green,26
Mazda,SUV,Red,29
Mazda,SUV,Blue,28
Mazda,SUV,Green,40
Mazda,Sedan,Red,36
Mazda,Sedan,Blue,48
Mazda,Sedan,Green,49
Mazda,Compact,Red,49
Mazda,Compact,Blue,37
Mazda,Compact,Green,50


In [4]:
data_file = "./carsales2-noHeader.txt"
# data imported without headers
raw_data = sc.textFile(data_file).cache()

In [ ]:
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

In [5]:
from pyspark.sql import Row
csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    Make = p[0],                            #  names of columns
    Style = p[1],
    Colour = p[2],
    Sales = int(p[3])
  ))

##Inner SQL Temp Table / First Dataframe

In [6]:
#CarSales_df = sqlContext.createDataFrame(row_data)
#CarSales_df.registerTempTable("CarSales2")
CarSales_df = spark.createDataFrame(row_data)
CarSales_df.createOrReplaceTempView("CarSales2")


In [7]:
CarSales_df.show()

+------+-------+------+-----+
|  Make|  Style|Colour|Sales|
+------+-------+------+-----+
| Honda|    SUV|   Red|   31|
| Honda|    SUV|  Blue|   46|
| Honda|    SUV| Green|   48|
| Honda|  Sedan|   Red|   25|
| Honda|  Sedan|  Blue|   41|
| Honda|  Sedan| Green|   34|
| Honda|Compact|   Red|   48|
| Honda|Compact|  Blue|   26|
| Honda|Compact| Green|   32|
|Toyota|    SUV|   Red|   39|
|Toyota|    SUV|  Blue|   45|
|Toyota|    SUV| Green|   50|
|Toyota|  Sedan|   Red|   41|
|Toyota|  Sedan|  Blue|   28|
|Toyota|  Sedan| Green|   21|
|Toyota|Compact|   Red|   44|
|Toyota|Compact|  Blue|   50|
|Toyota|Compact| Green|   26|
| Mazda|    SUV|   Red|   29|
| Mazda|    SUV|  Blue|   28|
+------+-------+------+-----+
only showing top 20 rows



In [8]:
#sqlContext.sql("SELECT * FROM CarSales2").show()
spark.sql("SELECT * FROM CarSales2").show()

+------+-------+------+-----+
|  Make|  Style|Colour|Sales|
+------+-------+------+-----+
| Honda|    SUV|   Red|   31|
| Honda|    SUV|  Blue|   46|
| Honda|    SUV| Green|   48|
| Honda|  Sedan|   Red|   25|
| Honda|  Sedan|  Blue|   41|
| Honda|  Sedan| Green|   34|
| Honda|Compact|   Red|   48|
| Honda|Compact|  Blue|   26|
| Honda|Compact| Green|   32|
|Toyota|    SUV|   Red|   39|
|Toyota|    SUV|  Blue|   45|
|Toyota|    SUV| Green|   50|
|Toyota|  Sedan|   Red|   41|
|Toyota|  Sedan|  Blue|   28|
|Toyota|  Sedan| Green|   21|
|Toyota|Compact|   Red|   44|
|Toyota|Compact|  Blue|   50|
|Toyota|Compact| Green|   26|
| Mazda|    SUV|   Red|   29|
| Mazda|    SUV|  Blue|   28|
+------+-------+------+-----+
only showing top 20 rows



In [9]:
#sums_df = sqlContext.sql("""
sums_df = spark.sql("""
  SELECT
    Make,
    SUM(IF(Style='SUV',Sales,0)) SUV,
    SUM(IF(Style='Sedan',Sales,0)) Sedan,
    SUM(IF(Style='Compact',Sales,0)) Compact
  FROM CarSales2
  GROUP BY Make WITH ROLLUP
""")
sums_df.show()

+------+---+-----+-------+
|  Make|SUV|Sedan|Compact|
+------+---+-----+-------+
|  null|356|  323|    362|
|Toyota|134|   90|    120|
| Honda|125|  100|    106|
| Mazda| 97|  133|    136|
+------+---+-----+-------+



##Outer SQL

In [10]:
#sums_df.registerTempTable("sums")
sums_df.createOrReplaceTempView("sums")
#CrossTab = sqlContext.sql("""
CrossTab = spark.sql("""
 SELECT
  IFNULL(Make,'#Totals') AS Dims,
  SUV, Sedan, Compact,
  SUV + Sedan + Compact AS Totals
 from sums
 order by Dims desc
""")
CrossTab.show()

+-------+---+-----+-------+------+
|   Dims|SUV|Sedan|Compact|Totals|
+-------+---+-----+-------+------+
| Toyota|134|   90|    120|   344|
|  Mazda| 97|  133|    136|   366|
|  Honda|125|  100|    106|   331|
|#Totals|356|  323|    362|  1041|
+-------+---+-----+-------+------+



#Super Store Data ( Large)

##Load Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [11]:
!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'

--2023-09-02 05:45:44--  https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
Resolving drive.google.com (drive.google.com)... 173.194.212.139, 173.194.212.101, 173.194.212.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.212.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fq4i8g16hnl1qo819p8t8dlhdop6ugak/1693633500000/15264228191300097767/*/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ?e=download&uuid=5f34ced3-7e88-4e86-bbfa-c6282727ef16 [following]
--2023-09-02 05:45:45--  https://doc-14-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fq4i8g16hnl1qo819p8t8dlhdop6ugak/1693633500000/15264228191300097767/*/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ?e=download&uuid=5f34ced3-7e88-4e86-bbfa-c6282727ef16
Resolving doc-14-5g-docs.googleusercontent.com (doc-14-5g-docs.googleusercontent.com)... 173.194.212.132, 

In [12]:
!head SS_Orders2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

##Clean Up

In [13]:
data_file = 'SS_Orders2.csv'
raw_data = sc.textFile(data_file)
print ("Data Size", raw_data.count())
#eCommerce_df = spark.read.csv(data_file,inferSchema=True, header=True)
Orders_df = spark.read.csv(data_file,inferSchema=True, header=True)

Data Size 9995


In [ ]:
#Orders2_df = spark.read.csv(data_file,inferSchema=True, header=True)

In [14]:
#eCommerce_df.printSchema()
Orders_df.printSchema()
# NOTE that everything is read as string ... this will not do!

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



In [15]:
!head SS_Orders2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

In [16]:
#we remove the CRLF character from the end of the row if it exists
!sed 's/\r//g' /content/SS_Orders2.csv > datafile.csv
# remove the first line containing headers from the file
#!sed -i -e "1d" datafile.csv
!head datafile.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United Stat

In [17]:
# remove , and " from data
import pandas as pd
p_df = pd.read_csv('datafile.csv')
#p_df = pd.read_csv('datafile.csv')
p_df2 = p_df.replace(',', ' ', regex=True)
p_df3 = p_df2.replace('"', '', regex=True)
p_df3.to_csv('datafile2.csv',index=False,sep=',')
!head datafile2.csv

Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs  Rounded Back,731.94,3,0.0,219.582
3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.0,6.8714
4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United 

In [18]:
data_file = 'datafile2.csv'
Orders2_df = spark.read.csv(data_file,inferSchema=True, header=True)

In [19]:
Orders2_df.printSchema()
# NOTE ... Sales, Quantity, Discount, Profit are now numeric

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [20]:
#Now convert dataframe into table called Orders
Orders2_df.createOrReplaceTempView("Orders")

In [21]:
Orders = spark.sql("SELECT * FROM Orders limit 20")
Orders.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+----------------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|          Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+----------------+
|     1|CA-2016-152156|08/11/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  S

#Super Store Data Processing

##Simple Queries

In [22]:
spark.sql("SELECT * FROM Orders limit 20").show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+----------------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|          Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+----------------+
|     1|CA-2016-152156|08/11/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  S

In [23]:
spark.sql("select count(Quantity) from Orders").show()

+---------------+
|count(Quantity)|
+---------------+
|           9994|
+---------------+



In [24]:
spark.sql("select Category, Sum(Sales), Sum(Profit) from Orders group by Category").show()

+---------------+-----------------+------------------+
|       Category|       sum(Sales)|       sum(Profit)|
+---------------+-----------------+------------------+
|Office Supplies|719047.0320000029|122490.80080000011|
|      Furniture|741999.7952999998|        18451.2728|
|     Technology|836154.0329999966| 145454.9480999999|
+---------------+-----------------+------------------+



In [25]:
spark.sql("select Region, Sum(Sales), Sum(Profit) from Orders group by Region").show()

+-------+-----------------+------------------+
| Region|       sum(Sales)|       sum(Profit)|
+-------+-----------------+------------------+
|  South|391721.9050000003| 46749.43030000006|
|Central|501239.8908000005| 39706.36249999998|
|   East|678781.2399999979| 91522.78000000026|
|   West|725457.8245000006|108418.44890000016|
+-------+-----------------+------------------+



In [26]:
spark.sql("select Segment, Sum(Sales), Sum(Profit) from Orders group by Segment").show()


+-----------+------------------+------------------+
|    Segment|        sum(Sales)|       sum(Profit)|
+-----------+------------------+------------------+
|   Consumer|1161401.3449999888|134119.20919999972|
|Home Office| 429653.1485000003| 60298.67850000009|
|  Corporate| 706146.3668000001| 91979.13400000021|
+-----------+------------------+------------------+



In [27]:
spark.sql("select Segment, Region, Sum(Sales) Sales from Orders group by Segment, Region").show()

+-----------+-------+------------------+
|    Segment| Region|             Sales|
+-----------+-------+------------------+
|Home Office|   West| 136721.7769999999|
|  Corporate|   West|225855.27449999977|
|   Consumer|   West| 362880.7730000003|
|   Consumer|   East|350908.16700000066|
|Home Office|Central| 91212.64399999994|
|  Corporate|Central|       157995.8128|
|  Corporate|  South|121885.93250000005|
|Home Office|  South| 74255.00150000004|
|   Consumer|  South|195580.97100000017|
|  Corporate|   East|200409.34699999995|
|Home Office|   East|127463.72599999998|
|   Consumer|Central|252031.43400000007|
+-----------+-------+------------------+



In [28]:
spark.sql("select Segment, Region, Sum(Sales) Sales from Orders group by Segment, Region with Rollup").show()

+-----------+-------+------------------+
|    Segment| Region|             Sales|
+-----------+-------+------------------+
|  Corporate|   null| 706146.3668000001|
|Home Office|Central| 91212.64399999994|
|   Consumer|Central|252031.43400000007|
|   Consumer|   West| 362880.7730000003|
|Home Office|   West| 136721.7769999999|
|  Corporate|  South|121885.93250000005|
|  Corporate|   West|225855.27449999977|
|       null|   null| 2297200.860299955|
|Home Office|   East|127463.72599999998|
|Home Office|  South| 74255.00150000004|
|  Corporate|Central|       157995.8128|
|   Consumer|  South|195580.97100000017|
|   Consumer|   East|350908.16700000066|
|   Consumer|   null|1161401.3449999888|
|  Corporate|   East|200409.34699999995|
|Home Office|   null| 429653.1485000003|
+-----------+-------+------------------+



##Cross Tabs

###Inner Tale

In [29]:
sums_df = spark.sql("""
    SELECT
        Segment,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='South',Sales,0)) As South
    FROM Orders
        GROUP BY Segment WITH ROLLUP
""")
sums_df.show()

+-----------+------------------+------------------+------------------+------------------+
|    Segment|           Central|              East|              West|             South|
+-----------+------------------+------------------+------------------+------------------+
|Home Office| 91212.64399999994|127463.72599999998| 136721.7769999999| 74255.00150000004|
|  Corporate|       157995.8128|200409.34699999995|225855.27449999977|121885.93250000005|
|       null| 501239.8908000005| 678781.2399999979| 725457.8245000006| 391721.9050000003|
|   Consumer|252031.43400000007|350908.16700000066| 362880.7730000003|195580.97100000017|
+-----------+------------------+------------------+------------------+------------------+



###Outer Query

In [30]:
#sums_df.registerTempTable("sums")
sums_df.createOrReplaceTempView("sums")
#CrossTab = sqlContext.sql("""
CrossTab = spark.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  sums.Central, sums.East, sums.West,sums.South,
  sums.Central + sums.East + sums.West + sums.South Totals
FROM sums
order by Dims desc
""")
CrossTab.show()

+-----------+------------------+------------------+------------------+------------------+------------------+
|       Dims|           Central|              East|              West|             South|            Totals|
+-----------+------------------+------------------+------------------+------------------+------------------+
|Home Office| 91212.64399999994|127463.72599999998| 136721.7769999999| 74255.00150000004| 429653.1484999999|
|  Corporate|       157995.8128|200409.34699999995|225855.27449999977|121885.93250000005| 706146.3667999997|
|   Consumer|252031.43400000007|350908.16700000066| 362880.7730000003|195580.97100000017|1161401.3450000011|
|    #Totals| 501239.8908000005| 678781.2399999979| 725457.8245000006| 391721.9050000003|2297200.8602999994|
+-----------+------------------+------------------+------------------+------------------+------------------+



In [31]:
CrossTab = spark.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  round(sums.Central,2) Central,
  round(sums.East,2) East,
  round(sums.West,2) West,
  round(sums.South,2) South,
  round(sums.Central + sums.East + sums.West + sums.South,2) Totals
FROM sums
order by Dims desc
""")
CrossTab.show()

+-----------+---------+---------+---------+---------+----------+
|       Dims|  Central|     East|     West|    South|    Totals|
+-----------+---------+---------+---------+---------+----------+
|Home Office| 91212.64|127463.73|136721.78|  74255.0| 429653.15|
|  Corporate|157995.81|200409.35|225855.27|121885.93| 706146.37|
|   Consumer|252031.43|350908.17|362880.77|195580.97|1161401.35|
|    #Totals|501239.89|678781.24|725457.82|391721.91|2297200.86|
+-----------+---------+---------+---------+---------+----------+



In [33]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Calcutta')))

Last tested   2023-09-02 11:18:44.955266+05:30


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)